In [91]:
import os
import random
import numpy as np
import glob
import torch
import torchvision
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
# from utils import *
from models import init_vgg, Mononito
from utils import *
# from utils import progress_bar

import random
import matplotlib.pyplot as plt
%matplotlib inline
from numpy import moveaxis
import sys

from sklearn.metrics import accuracy_score

device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [92]:
class ZO_AdaMM(object):
    """
    Black Box optimizer
    """
    
    def __init__(self,inputVar,func,timesteps=1,alpha=None,beta_1=None,beta_2=None,m=None,v=None,
                 v_app=None,device='cpu'):
        self.inputVar=inputVar
        self.orgShape=self.inputVar.shape
        
        if len(self.orgShape)!=0:
            self.inputVar=torch.flatten(self.inputVar)
            self.dim=np.prod(self.orgShape)
            self.scalar=False
        else:
            self.scalar=True
        
        try:
            if not torch.is_tensor(self.inputVar):
                raise TypeError
        except TypeError:
            print("Input should be a tensor")
         
        #self.inputVar.requires_grad_()
        self.func=func
        
        self.args={'timesteps':timesteps,'alpha':alpha,'beta_1':beta_1,'beta_2':beta_2,
                   'm':m,'v':v,'v_app':v_app}
        
        self.timesteps=timesteps
        self.mu=1.0
        
        if alpha==None:
            self.alpha=np.random.uniform(1e-3,1e-2,size=(self.timesteps,))
            
        if beta_1==None:
            self.beta_1=np.random.uniform(1e-3,1e-2,size=(self.timesteps,))
        
        if beta_2==None:
            self.beta_2=np.random.uniform(0,1)
            
        if m==None:
            if not self.scalar:
                self.m=torch.zeros(self.dim).double().to(device)
            else:
                self.m=torch.tensor(0.0).to(device)
        if v==None:
            if not self.scalar:
                self.v=torch.zeros(self.dim).double().to(device)
            else:
                self.v=torch.tensor(0.0).to(device)
        
        if v_app==None:
            if not self.scalar:
                self.v_app=torch.zeros(self.dim).double().to(device)
            else:
                self.v_app=torch.tensor(0.0).to(device)
        
        if not self.scalar:
            self.grad=torch.zeros(self.dim).double().to(device)
        else:
            self.grad=torch.tensor(0.0).to(device)
        
    def step(self):
        for t in range(self.timesteps):
            if not self.scalar:
                u=torch.tensor(np.random.uniform(0,1,self.dim)).double().to(device)
            else:
                u=torch.tensor(np.random.uniform()).to(device)
            
            if not self.scalar:
                g_t= (self.func((self.inputVar+self.mu*u).reshape(self.orgShape))-
                      self.func(self.inputVar.reshape(self.orgShape)))*(self.dim/self.mu)*u
#                 print(g_t.dtype)
            else:
                g_t= (self.func(self.inputVar+self.mu*u)-self.func(self.inputVar))*(1.0/self.mu)*u
            
            self.m=self.beta_1[t]*self.m + (1.0-self.beta_1[t])*g_t
            self.v=self.beta_2*self.v + (1.0-self.beta_2)*(g_t**2)
            
            self.v_app=torch.max(self.v_app,self.v)
            
            if not self.scalar:
                if device=='cpu':
                    v_app_np= np.diag(self.v_app.flatten().detach().numpy())
                else:
                    v_app_np= np.diag(self.v_app.flatten().cpu().detach().numpy())

            else:
                v_app_np=self.v_app.numpy()
            
            
            if not self.scalar:
                if device=='cpu':
                    self.grad= torch.tensor(np.matmul(self.alpha[t]*linalg.fractional_matrix_power(v_app_np,-0.5),
                                                      self.m.flatten().detach().numpy())).to(device)
                else:
                    self.grad= torch.tensor(np.matmul(self.alpha[t]*linalg.fractional_matrix_power(v_app_np,-0.5),
                                                      self.m.flatten().cpu().detach().numpy())).to(device)
            else:
                self.grad=torch.tensor(self.alpha[t]*np.power(v_app_np,-0.5)*self.m.numpy()).to(device)
            
            self.inputVar=self.inputVar-self.grad
            
    
    def zero_grad(self):
        self.__init__(self.inputVar,self.func,**self.args)
    
    def returnInput(self):
        if not self.scalar:
            return self.inputVar.reshape(self.orgShape)
        else:
            return self.inputVar
            
    def gradFn(self):
        return self.grad.reshape(self.orgShape)

In [93]:
WEIGHT_PATH='./Model_weights/'
TEST_PATH='./data/v2_cam1_cam2_ split_by_driver/Camera 1/train'
classes=["c"+ str(i) for i in range(10)]
# classes.index("c3")

In [94]:
manualseed=random.randint(1,10000)
random.seed(manualseed)
np.random.seed(manualseed)
torch.manual_seed(manualseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(manualseed)

cudnn.benchmark = True

In [95]:
vggModel=init_vgg()
vggWeights=torch.load(WEIGHT_PATH+'vgg.pt')
vggModel.load_state_dict(vggWeights)
vggModel.eval()


Sequential(
  (0): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16): M

In [96]:
mononitoModel=Mononito()
mononitoWeights=torch.load(WEIGHT_PATH+'are_you_paying_attention.pt')
mononitoModel.load_state_dict(mononitoWeights)
mononitoModel.eval()

Mononito(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.3, inplace=False)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (dropout2): Dropout(p=0.3, inplace=False)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (relu3): ReLU()
  (maxpool3): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (dropout3): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1024, out_features=192, bias=True)
  (dropout4): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=192, out_features=10, bias=True)
)

In [97]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=1)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=1)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [98]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.train()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [99]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 2.216
[1,  4000] loss: 1.885
[1,  6000] loss: 1.672
[1,  8000] loss: 1.589
[1, 10000] loss: 1.519
[1, 12000] loss: 1.469
Finished Training


In [100]:
net.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 48 %


In [112]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=False, num_workers=1)
test_loader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=1)

In [113]:
# len(testset)

In [114]:
min_in, max_in = 32, 32
min_in, max_in = np.array([min_in, min_in, min_in]), np.array([max_in, max_in, max_in])
mean, std = np.array(0.5), np.array(0.5) 
min_out, max_out = np.min((min_in-mean)/std), np.max((max_in-mean)/std)

In [115]:
# Variables to define

target = 0
conf_target = 0.9
max_count = 100
patch_type = 'circle'
patch_size = 0.05
image_size = 32
train_size = 50000
test_size = 10000
plot_all = 1
outf='./tmp'
# logfile='./tmp/log.txt'
zoAdamm=True

In [116]:
# device='cpu'

In [117]:
net.double().to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [118]:
def train(classifier,train_loader, epoch, patch, patch_shape):
    classifier.eval()
    success = 0
    total = 0
    recover_time = 0
    for batch_idx, (data, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            data = data.double().to(device)
#             print(data.dtype)
            labels = labels.to(device)
        data, labels = Variable(data), Variable(labels)

        prediction = classifier(data)
 
        # only computer adversarial examples on examples that are originally classified correctly        
        if prediction.data.max(1)[1][0] != labels.data[0]:
            continue
     
        total += 1
        
        # transform path
        data_shape = data.data.cpu().numpy().shape
        if patch_type == 'circle':
            patch, mask, patch_shape = circle_transform(patch, data_shape, patch_shape, image_size)
        elif patch_type == 'square':
            patch, mask  = square_transform(patch, data_shape, patch_shape, image_size)
        patch, mask = torch.FloatTensor(patch), torch.FloatTensor(mask)
        if torch.cuda.is_available():
            patch, mask = patch.to(device), mask.to(device)
        patch, mask = Variable(patch), Variable(mask)
 
        adv_x, mask, patch = attack(classifier, data, patch, mask)
        
        adv_label = classifier(adv_x).data.max(1)[1][0]
        ori_label = labels.data[0]
        
        if adv_label == target:
            success += 1
      
            if plot_all == 1: 
                # plot source image
                vutils.save_image(data.data, "./%s/%d_%d_original.png" %(outf, batch_idx, ori_label), normalize=True)
                
                # plot adversarial image
                vutils.save_image(adv_x.data, "./%s/%d_%d_adversarial.png" %(outf, batch_idx, adv_label), normalize=True)
 
        masked_patch = torch.mul(mask, patch)
        patch = masked_patch.data.cpu().numpy()
        new_patch = np.zeros(patch_shape)
        for i in range(new_patch.shape[0]): 
            for j in range(new_patch.shape[1]): 
                new_patch[i][j] = submatrix(patch[i][j])
 
        patch = new_patch

        # log to file  
        progress_bar(batch_idx, len(train_loader), "Train Patch Success: {:.3f}".format(success/total))

    return patch

def test(classifier,test_loader, epoch, patch, patch_shape):
    classifier.eval()
    success = 0
    total = 0
    for batch_idx, (data, labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            data = data.double().to(device)
            labels = labels.to(device)
        data, labels = Variable(data), Variable(labels)

        prediction = classifier(data)

        # only computer adversarial examples on examples that are originally classified correctly        
        if prediction.data.max(1)[1][0] != labels.data[0]:
            continue
      
        total += 1 
        
        # transform path
        data_shape = data.data.cpu().numpy().shape
        if patch_type == 'circle':
            patch, mask, patch_shape = circle_transform(patch, data_shape, patch_shape, image_size)
        elif patch_type == 'square':
            patch, mask = square_transform(patch, data_shape, patch_shape, image_size)
        patch, mask = torch.FloatTensor(patch), torch.FloatTensor(mask)
        if torch.cuda.is_available():
            patch, mask = patch.to(device), mask.to(device)
        patch, mask = Variable(patch), Variable(mask)
 
        adv_x = torch.mul((1-mask),data) + torch.mul(mask,patch)
        adv_x = torch.clamp(adv_x, min_out, max_out)
        
        adv_label = classifier(adv_x).data.max(1)[1][0]
        ori_label = labels.data[0]
        
        if adv_label == target:
            success += 1
       
        masked_patch = torch.mul(mask, patch)
        patch = masked_patch.data.cpu().numpy()
        new_patch = np.zeros(patch_shape)
        for i in range(new_patch.shape[0]): 
            for j in range(new_patch.shape[1]): 
                new_patch[i][j] = submatrix(patch[i][j])
 
        patch = new_patch

        # log to file  
        progress_bar(batch_idx, len(test_loader), "Test Success: {:.3f}".format(success/total))

def attack(classifier,x, patch, mask):
    classifier.eval()

    x_out = F.softmax(classifier(x))
    target_prob = x_out.data[0][target]
    
    patch=patch.double()
    mask=mask.double()
    adv_x = (torch.mul((1-mask),x) + torch.mul(mask,patch)).double()
#     adv_x=adv_x.double()
#     print(adv_x.dtype)
    
    count = 0 
   
    while conf_target > target_prob:
        count += 1
        adv_x = Variable(adv_x.data, requires_grad=True)
        
        if not zoAdamm:
            adv_out = F.log_softmax(classifier(adv_x))
       
            adv_out_probs, adv_out_labels = adv_out.max(1)
        
            Loss = -adv_out[0][target]
            Loss.backward()
     
            adv_grad = adv_x.grad.clone()
        
            adv_x.grad.data.zero_()
       
            patch -= adv_grad 
        else:
#             adv_x=adv_x.double()
            def func(adv_input):
#                 print(adv_input.dtype)
                adv_out = F.log_softmax(classifier(adv_input))
                Loss = -adv_out[0][target]
                return Loss

            opt=ZO_AdaMM(adv_x,func,timesteps=1,device=device)
#             print(opt)
            opt.step()
#             print(opt.grad())
            patch-=opt.gradFn()
            
        
        adv_x = (torch.mul((1-mask),x) + torch.mul(mask,patch)).double()
        adv_x = torch.clamp(adv_x, min_out, max_out)
 
        out = F.softmax(classifier(adv_x))
        target_prob = out.data[0][target]
        #y_argmax_prob = out.data.max(1)[0][0]
        
        #print(count, conf_target, target_prob, y_argmax_prob)  

        if count >= max_count:
            break


    return adv_x, mask, patch 

In [119]:
# x=np.random.rand(1,3,4)
# np.prod(x.shape)
# z=torch.rand(1,2,3).flatten()
# x=torch.FloatTensor(np.random.rand(1,3,32,32))
# net(x)

In [120]:
patch,patch_shape = init_patch_circle(image_size,patch_size)
for epoch in range(1,2):
    patch=train(net,train_loader,epoch,patch,patch_shape)
    test(net,test_loader,epoch,patch,patch_shape)

KeyboardInterrupt: 